# Process Existing YOLO Annotations into PascalVOC XML

In [7]:
!dir

 Volume in drive D is sandisk h3
 Volume Serial Number is 4ED5-B0C0

 Directory of d:\code\_projects\_practice\TFODCourse-main

03/04/2021  09:26 AM    <DIR>          .
03/04/2021  09:26 AM    <DIR>          ..
03/04/2021  09:26 AM                75 .gitignore
08/09/2022  01:26 AM             8,795 1. Image Collection.ipynb
09/09/2022  02:04 AM           117,381 2. Training and Detection.ipynb
03/04/2021  09:26 AM             2,988 Error Guide.md
03/04/2021  09:26 AM             3,132 README.md
03/04/2021  09:26 AM    <DIR>          .ipynb_checkpoints
08/09/2022  12:07 AM               735 han.xml
08/09/2022  12:17 AM    <DIR>          plant_pathology_2021_fgvc8
08/09/2022  12:34 AM    <DIR>          flowers
08/09/2022  12:36 AM               127 source.txt
08/09/2022  01:01 AM    <DIR>          fruitssmall
08/09/2022  01:02 AM    <DIR>          vehicle
08/09/2022  01:11 AM             1,016 install_guide_venv_python_tf.md
08/09/2022  02:28 AM    <DIR>          Tensorflow
09/09/2022  0

In [4]:
import os
import cv2 as cv
import xml.etree.ElementTree as ET

paths = {
    'images' : r'.\Tensorflow\workspace\safetyvest\train',
    'labels' : r'.\Tensorflow\workspace\safetyvest\train',
    # 'class1'    : 'axial',
    # 'class2'    : 'coronal',
    # 'class3'    : 'sagittal',   
}

# trainclass1 = os.listdir(os.path.join(paths['workspace'], paths['class1'], 'images', 'train'))
# trainclass2 = os.listdir(os.path.join(paths['workspace'], paths['class2'], 'images', 'train'))
# trainclass3 = os.listdir(os.path.join(paths['workspace'], paths['class3'], 'images', 'train'))
# testclass1 = os.listdir(os.path.join(paths['workspace'], paths['class1'], 'images', 'test'))
# testclass2 = os.listdir(os.path.join(paths['workspace'], paths['class2'], 'images', 'test'))
# testclass3 = os.listdir(os.path.join(paths['workspace'], paths['class3'], 'images', 'test'))

In [1]:
def compare_txtannot_against_images(path_img,path_txt):
    # path_img = os.path.join(path, 'images')
    # path_txt = os.path.join(path, 'labels')
    files_img = os.listdir(path_img)
    files_img = [f for f in files_img if f[-3:]=='jpg']
    files_txt = os.listdir(path_txt)
    files_txt = [f for f in files_txt if f[-3:]=='txt']
    matches = []
    
    for i in files_img:
        for t in files_txt:
            if (i[:-4]==t[:-4]):
                matches.append(i)
    
    nonmatch_img = [x for x in files_img if x not in matches]
    nonmatch_txt = [x for x in files_txt if x not in matches]

    return nonmatch_img, nonmatch_txt


    
    


def retrieve_annotation(curr_image_path, curr_label_path):
    img = cv.imread(curr_image_path)
    img_width  = img.shape[1]
    img_height = img.shape[0]
    img_depth = img.shape[2]
    # print(img_width, img_height)

    with open(curr_label_path) as labeltxt:
        labeltxt = labeltxt.read()
        labeltxt = labeltxt.split()
        # print(labeltxt)
        label = labeltxt[0]
        x = labeltxt[1]
        y = labeltxt[2]
        w = labeltxt[3]
        h = labeltxt[4]
        w = round(float(w) * img_width)
        h = round(float(h) * img_height)
        x = round((float(x) * img_width) - w/2)
        y = round((float(y) * img_height) - h/2)

        # print(x, y, w, h)

        # cv.rectangle(img, (int(x),int(y)), (int(x+w),int(y+h)), (0,255,0), thickness=2)
        # cv.imshow('Image', img)
        # cv.waitKey(0)
        return x,y,w,h,img_width,img_height,label

def save_annotation(x,y,w,h,img_width,img_height,label, curr_image, curr_image_path, labelprefix, folderprefix):
    
    annotation = ET.Element('annotation')

    #! REM TO SET
    folder = ET.SubElement(annotation, 'folder')
    folder.text = str(folderprefix)

    filename = ET.SubElement(annotation, 'filename')
    filename.text = curr_image

    path = ET.SubElement(annotation, 'path')
    path.text = curr_image_path

    source  = ET.SubElement(annotation, 'source')
    database = ET.SubElement(source, 'database')
    database.text = 'Unknown'

    size  = ET.SubElement(annotation, 'size')

    width   = ET.SubElement(size, 'width')
    width.text = str(img_width)
    height  = ET.SubElement(size, 'height')
    height.text = str(img_height)
    depth  = ET.SubElement(size, 'depth')
    depth.text = str(img_depth)

    segmented  = ET.SubElement(annotation, 'segmented')
    segmented.text= '0'


    object  = ET.SubElement(annotation, 'object')
    name = ET.SubElement(object, 'name')
    name.text = labelprefix + str(label)

    pose = ET.SubElement(object, 'pose')
    pose.text = 'Unspecified'

    truncated = ET.SubElement(object, 'truncated')
    truncated.text = '0'

    difficult = ET.SubElement(object, 'difficult')
    difficult.text = '0'

    bndbox = ET.SubElement(object, 'bndbox')
    xmin = ET.SubElement(bndbox, 'xmin')
    ymin = ET.SubElement(bndbox, 'ymin')
    xmax = ET.SubElement(bndbox, 'xmax')
    ymax = ET.SubElement(bndbox, 'ymax')
    xmin.text = str(x)
    ymin.text = str(y)
    xmax.text = str(x+w)
    ymax.text = str(y+h)

    b_xml = ET.tostring(annotation)
    
    xml_path = curr_image_path[:-4] + '.xml'

    with open(xml_path, "wb") as f:
        f.write(b_xml)

def process_annotations(path_images, path_labels, labelprefix, folderprefix):
    filelist = os.listdir(path_images)

    for curr_image in filelist[:3]:
        curr_label = curr_image.split('.')[0] + '.txt'

        curr_image_path = os.path.join(path_images, curr_image)
        curr_label_path = os.path.join(path_labels, curr_label)
        # print(curr_image_path, curr_label_path)

        if os.path.exists(curr_image_path) & os.path.exists(curr_label_path) & ('jpg' in curr_image_path):
            x,y,w,h,img_w,img_h,label = retrieve_annotation(curr_image_path, curr_label_path)
            save_annotation(x,y,w,h,img_w,img_h,label, curr_image, curr_image_path, labelprefix, folderprefix)



In [7]:
# path_images = os.path.join(paths['workspace'], paths['class1'], 'images', 'train')
# path_labels = os.path.join(paths['workspace'], paths['class1'], 'labels', 'train')

compare_txtannot_against_images(paths['images'], paths['labels'])

([],
 ['pos_1950.txt',
  'pos_1949.txt',
  'pos_1948.txt',
  'pos_1947.txt',
  'pos_1946.txt',
  'pos_1945.txt',
  'pos_1944.txt',
  'pos_1943.txt',
  'pos_1942.txt',
  'pos_1941.txt',
  'pos_1940.txt',
  'pos_1939.txt',
  'pos_1938.txt',
  'pos_1937.txt',
  'pos_1936.txt',
  'pos_1935.txt',
  'pos_1934.txt',
  'pos_1933.txt',
  'pos_1932.txt',
  'pos_1931.txt',
  'pos_1930.txt',
  'pos_1929.txt',
  'pos_1928.txt',
  'pos_1927.txt',
  'pos_1926.txt',
  'pos_1925.txt',
  'pos_1924.txt',
  'pos_1923.txt',
  'pos_1922.txt',
  'pos_1921.txt',
  'pos_1920.txt',
  'pos_1919.txt',
  'pos_1918.txt',
  'pos_1917.txt',
  'pos_1916.txt',
  'pos_1915.txt',
  'pos_1914.txt',
  'pos_1913.txt',
  'pos_1912.txt',
  'pos_1911.txt',
  'pos_1910.txt',
  'pos_1909.txt',
  'pos_1908.txt',
  'pos_1907.txt',
  'pos_1906.txt',
  'pos_1905.txt',
  'pos_1904.txt',
  'pos_1903.txt',
  'pos_1902.txt',
  'pos_1901.txt',
  'pos_1900.txt',
  'pos_1899.txt',
  'pos_1898.txt',
  'pos_1897.txt',
  'pos_1896.txt',
  'po

In [ ]:

    # printing the attributes of the
    # first tag from the parent
    # print(root[0].attrib)
    
    # printing the text contained within
    # first subtag of the 5th tag from
    # the parent
    # print(root[5][0].text)



# with open(template) as template:
#     template = template.read()
#     # print(template.annotations)

#     template = BeautifulSoup(template, 'xml')
 
#     # A loop for replacing the value
#     # of attribute `test` to WHAT !!
#     # The tag is found by the clause
#     # `bs_data.find_all('child', {'name':'Frank'})`
#     # for tag in template.find_all('child', {'name':'Frank'}):
#     #     tag['test'] = "WHAT !!"
    
    
#     # Output the contents of the
#     # modified xml file
#     print(template.prettify())

In [6]:
process_annotations(path_images, path_labels, labelprefix='axial', folderprefix='test')

path_images = os.path.join(paths['workspace'], paths['class1'], 'images', 'train')
path_labels = os.path.join(paths['workspace'], paths['class1'], 'labels', 'train')
process_annotations(path_images, path_labels, labelprefix='axial', folderprefix='train')

path_images = os.path.join(paths['workspace'], paths['class2'], 'images', 'test')
path_labels = os.path.join(paths['workspace'], paths['class2'], 'labels', 'test')
process_annotations(path_images, path_labels, labelprefix='coronal', folderprefix='test')

path_images = os.path.join(paths['workspace'], paths['class2'], 'images', 'train')
path_labels = os.path.join(paths['workspace'], paths['class2'], 'labels', 'train')
process_annotations(path_images, path_labels, labelprefix='coronal', folderprefix='train')

path_images = os.path.join(paths['workspace'], paths['class3'], 'images', 'test')
path_labels = os.path.join(paths['workspace'], paths['class3'], 'labels', 'test')
process_annotations(path_images, path_labels, labelprefix='sagittal', folderprefix='test')

path_images = os.path.join(paths['workspace'], paths['class3'], 'images', 'train')
path_labels = os.path.join(paths['workspace'], paths['class3'], 'labels', 'train')
process_annotations(path_images, path_labels, labelprefix='sagittal', folderprefix='train')



NameError: name 'img_depth' is not defined